In [7]:
import nltk
import re
import pandas as pd
import numpy as np

#reading the data set - news data sets

data_set = pd.read_csv("news_summary.csv")
print(data_set.columns)
data_set
len(data_set)

Index(['headlines', 'text'], dtype='object')


98401

In [8]:
# Dividing Dataset into X and Y
text_review = data_set.drop(columns = 'headlines' , axis = 1)
text_review = text_review.iloc[:5000,:]
head_lines = data_set.drop(columns = 'text', axis = 1)
head_lines = head_lines.iloc[:5000,:]

In [9]:
# Text Preprocessing

#punctuations removal
p_text = []        #
p_headlines = []
for content in text_review.text:
    cleaned_text = re.sub(r'\[[0-9]*\]',' ',content)
    final_text=re.sub(r'\s+',' ',cleaned_text)
    final_text=re.sub(r'\[[a-zA-Z]*\]',' ',final_text)
    final_text=re.sub(r'\s+',' ',final_text)
    final_text=re.sub(r'\(([^)]*)\)',' ',final_text)
    final_text=re.sub(r'\s+',' ',final_text)
    p_text.append(final_text)
    


In [10]:
#contractions
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [12]:
#stop word removal and contractions replacing
sw=nltk.corpus.stopwords.words("english")
text_final = []
text_headlines = []
for text_1 in p_text:
    temp = text_1.split()
    new_text = []
    for word in temp:
        word = word.lower()
        if word in contractions.keys():
            new_text.append(contractions[word])
        else:
            new_text.append(word)
    t = " ".join(new_text)
    text_final.append(t)
for text_2 in p_headlines:
    temp = text_2.split()
    new_text1 = []
    for word in temp:
        word = word.lower()
        if word in contractions.keys():
            new_text1.append(contractions[word])
        else:
            new_text1.append(word)
    t1 = " ".join(new_text1)
    text_headlines.append(t1)

In [15]:
#counting the number of words and sentences in the text
word_count = {}
for sentence in text_final:
  for word in nltk.word_tokenize(sentence):
    if word in word_count.keys():
      word_count[word] += 1
    else:
      word_count[word] = 1

for sentence in text_headlines:
  for word in nltk.word_tokenize(sentence):
    if word in word_count.keys():
      word_count[word]+=1
    else:
      word_count[word] = 1
print(len(word_count.keys()))

24197
